**DSP DEMIXING**


In [ ]:
# Install the required packages
%pip install -r requirements.txt

Let's start with loading the dataset. The folder "musdbhq_trimmed" contains 30 seconda of all the tracks. Since we noticed that not all the stems of the tracks were non-silent in the first 30 seconds, we trimmed the dataset in order to retrieve 30 seconds of each track where every stem is non-silent, in order to have a more accurate measure.

In [ ]:
import tqdm, os, torchaudio

def load_dataset():
    """
    Load the dataset from the musdb18hq_trimmed folder.
    Each subfolder in the dataset corresponds to a song.
    Each song contains multiple stems (e.g., mixture, drums, bass, etc.).
    Returns:
        dataset_dict (dict): A dictionary where keys are track folders and values are dictionaries of stems.
    """
    dataset_dict = {}

    for track_folder in tqdm.tqdm(os.listdir("/Users/alessandromanattini/Desktop/MAE/SELECTED TOPIC/PROJECT STMAE/musdb18hq_trimmed")):
        track_path = os.path.join("/Users/alessandromanattini/Desktop/MAE/SELECTED TOPIC/PROJECT STMAE/musdb18hq_trimmed", track_folder)
        if not os.path.isdir(track_path):
            continue

        # Prepare a sub-dictionary for this song
        stems_dict = {}
        
        for stem_name in ["mixture", "drums", "bass", "vocals", "other", "new_mixture"]:
            file_path = os.path.abspath(os.path.join(track_path, f"{stem_name}.wav"))
            
            if not os.path.isfile(file_path):
                print(f"Warning: file not found {file_path}")
                continue

            # Load full audio
            waveform, sr = torchaudio.load(file_path)

            stems_dict[stem_name] = waveform
            
        dataset_dict[track_folder] = stems_dict
        
    return dataset_dict

In [ ]:
# Load the dataset
dataset_dict = load_dataset()  

print("Number of keys in dataset_dict:", len(dataset_dict))

# Check the first track folder and its contents
first_track_folder = list(dataset_dict.keys())[0]
print("First track folder:", first_track_folder)
print("Contents of the first track folder:")
for stem_name in dataset_dict[first_track_folder].keys():
    print(f" - {stem_name}: {dataset_dict[first_track_folder][stem_name].shape}")

Let's load all the mixtures in a list ***mixture_files[]***.


In [ ]:
# Load all new_mixture.wav files 
mixture_files = []
for track_folder in dataset_dict.keys():
    new_mixture_path = os.path.join("/Users/alessandromanattini/Desktop/MAE/SELECTED TOPIC/PROJECT STMAE/musdb18hq_trimmed", track_folder, "new_mixture.wav")
    if os.path.isfile(new_mixture_path):
        mixture_files.append(new_mixture_path)
    else:
        print(f"Warning: file not found {new_mixture_path}")


Define the parameters of the STFT:

In [ ]:
import librosa
# STFT parameters
n_fft = 2048
hop_length = 512
win = 'hann'

# Initialize lists to store STFT results
S_full_list = []
phase_list = []

# Loop through each mixture file and compute STFT
for mixture_path in mixture_files:
    # Carica l'audio dal file
    audio, sr = librosa.load(mixture_path, sr=None)
    # Calcola STFT
    D = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length, window=win)
    # Estrai modulo e fase
    mag, phase = librosa.magphase(D)
    
    S_full_list.append(mag)
    phase_list.append(phase)

Now we are going to define some functions:
- drumExtraction
- vocalExtraction
- bassOtherExtraction


**DRUMS EXTRACTION**
(using HPSS):

- STFT Magnitude Input: The function receives the magnitude (mix_mag) and phase (mix_phase) of the mixture’s STFT.

- HPSS Decomposition: It utilizes the **Harmonic-Percussive Source Separation (HPSS)** algorithm to split the mixture’s magnitude into two components:
    1) A ***harmonic component*** that captures the tonal content.
    2) A ***percussive component*** that emphasizes transient, drum-like features.

- Drums Reconstruction: The function then reconstructs the time-domain drums signal by combining the percussive component with the original phase information using the iSTFT.

- Output: The result is a time-domain signal (drums) that represents the extracted percussive (drum) elements from the mixture.

In [ ]:
# Drums extraction using HPSS
import numpy as np

def drums_extraction(mix_mag, mix_phase, n_fft=2048, hop_length=512):
    """
    Extract drums from a mixture using Harmonic-Percussive Source Separation (HPSS).
    
    Parameters:
        mixture (ndarray): The audio mixture.
        sr (int): Sample rate of the audio.
        n_fft (int): FFT window size.
        hop_length (int): Hop length for STFT.
        
    Returns:
        drums (ndarray): Extracted drums.
    """
    # Decompose the mixture into harmonic and percussive components
    S_harmonic, S_percussive = librosa.decompose.hpss(mix_mag)

    # Reconstruct the drums using the percussive component and the original phase
    y_drums = librosa.istft(S_percussive*mix_phase, hop_length=hop_length, win_length=n_fft, window=win)
    
    
    return y_drums, S_harmonic

**Vocal Extraction** (using REPET-SIM):

- STFT Magnitude Input: Similar to the drums extraction, this function takes the mixture’s magnitude (mix_mag) and phase (mix_phase) as input.

- NN Filter Processing: The function applies a ***nearest-neighbor (nn) filter*** to the magnitude. This filter:
    - Uses a median aggregation to estimate a smooth background signal.
    - Operates with a cosine similarity metric and a time window (converted from 2.0 seconds into frames) to capture repeating patterns.

- Filter Application: The filtered version (S_filter) is then constrained by taking the element-wise minimum with the original magnitude, ensuring that only components present in both are retained.

- Soft Mask Creation: A soft mask is computed using librosa.util.softmask that emphasizes differences between the original magnitude and the filtered background. This mask is tuned (with a factor of 100 and power 2) to highlight vocal components.

- Vocals Reconstruction: The function applies this mask to the original magnitude to produce a modified magnitude focused on the vocal content. It then reconstructs the time-domain vocal signal by combining this modified magnitude with the original phase via the iSTFT.

- Output: The final output is a time-domain signal representing the extracted vocals from the mixture.

In [ ]:
import warnings

# Vocal extraction using REPET-SIM
def vocal_extraction(mix_mag, mix_phase, n_fft=2048, hop_length=512):
    """
    Extract vocals from a mixture using REPET-SIM.
    """
    # add small epsilon to avoid zero‐vectors in cosine similarity
    eps = 1e-8
    mix_mag_eps = mix_mag + eps

    # suppress divide‐by‐zero / overflow warnings during nn_filter
    
    S_filter = librosa.decompose.nn_filter(
        mix_mag_eps,
        aggregate=np.median,
        metric='cosine',
        width=int(librosa.time_to_frames(2.0, sr=sr))
    )
    S_filter = np.minimum(S_filter, mix_mag_eps)
    S_filter = np.nan_to_num(S_filter, nan=0.0, posinf=0.0, neginf=0.0)

    # create and sanitize soft mask
    mask_vocal = librosa.util.softmask(
        mix_mag_eps - S_filter,
        100 * S_filter,
        power=2
    )
    mask_vocal = np.nan_to_num(mask_vocal, nan=0.0, posinf=0.0, neginf=0.0)

    S_vocal = mask_vocal * mix_mag_eps
    y_vocals = librosa.istft(
        S_vocal * mix_phase,
        hop_length=hop_length,
        win_length=n_fft,
        window=win
    )

    return y_vocals, mask_vocal

**BASS & OTHERS EXTRACTION**

For bass and other non-vocal components, the process begins with isolating the non-vocal harmonic residue by subtracting the estimated vocal contribution from the harmonic component. Frequency information is then used to create masks: one that selects frequencies below a certain threshold (250 Hz) to capture bass elements, and a complementary mask that selects the remaining frequencies for other elements. These masks are applied to the non-vocal residue, and by incorporating the original phase data, two distinct time-domain signals are reconstructed—one corresponding to the bass and the other to the rest of the non-vocal components.

In [ ]:
# Bass and Other extraction
def bass_other_extraction(S_harmonic, mask_vocal, n_fft=2048, hop_length=512):
    """
    Extract bass and other components from a mixture using a simple thresholding method.
    
    Parameters:
        S_harmonic (ndarray): The harmonic component of the mixture.
        mask_vocal (ndarray): The vocal mask obtained from the vocal extraction.
        n_fft (int): FFT window size.
        hop_length (int): Hop length for STFT.  
    Outputs:
        y_bass (ndarray): Extracted bass component.
        y_other (ndarray): Extracted other component.   
    """
    # Residuo armonico non-vocale
    S_resid = S_harmonic - (mask_vocal * S_harmonic)

    # Opzione A: filtro passa-basso per il basso (< 250 Hz)
    freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
    bass_mask = (freqs[:, None] < 250.0).astype(float)
    other_mask = 1.0 - bass_mask

    y_bass  = librosa.istft((S_resid * bass_mask) * phase, hop_length=hop_length, window=win)
    y_other = librosa.istft((S_resid * other_mask) * phase, hop_length=hop_length, window=win)

    return y_bass, y_other

Now we are going to do the separation for just the first track of the dataset to see the results.

In [ ]:
import IPython.display as ipd

# Perform extraction for the first mixture and listen to the results
mixture_path = mixture_files[0]
mixture, sr = librosa.load(mixture_path, sr=None)


# Select the first STFT result
mag_first_track = S_full_list[0]
phase_first_track = phase_list[0]

# Extract drums
y_drums, S_harmonic = drums_extraction(mag_first_track, phase_first_track, n_fft=n_fft, hop_length=hop_length)
# Extract vocals
y_vocals, mask_vocal = vocal_extraction(mag_first_track, phase_first_track, n_fft=n_fft, hop_length=hop_length)
# Extract bass and other
y_bass, y_other = bass_other_extraction(S_harmonic, mask_vocal, n_fft=n_fft, hop_length=hop_length)

# Listen to the results

print("Drums:")
display(ipd.Audio(y_drums, rate=sr))
print("Vocals:")
display(ipd.Audio(y_vocals, rate=sr))
print("Bass:")
display(ipd.Audio(y_bass, rate=sr))
print("Other:")
display(ipd.Audio(y_other, rate=sr))


In [ ]:
import librosa
# Load the first mixture of the dataset and play it
mixture, sr = librosa.load(mixture_path, sr=None)
print("Mixture:")
ipd.Audio(mixture, rate=sr)


In [ ]:
# hpss_stereo.py
# HPSS “from scratch” su segnali stereo

from __future__ import annotations
import numpy as np
import librosa
import scipy.ndimage
import soundfile as sf
from pathlib import Path


def hpss_stereo(
    path_in: str,
    *,
    sr: int = 44_100,
    n_fft: int = 4096,
    hop: int = 1024,
    harmonic_filt: int = 31,
    percussive_filt: int = 31,
    mask: str = "soft",        # "soft" oppure "binary"
    power: float = 2.0,        # =2 ⇒ spettro di potenza; =1 ⇒ magnitudine
    out_harm: str | None = None,
    out_perc: str | None = None,
) -> tuple[np.ndarray, np.ndarray]:
    """
    HPSS per file stereo: restituisce due array (shape=(2, n_samples))
    contenenti i segnali armonico e percussivo per i canali [L, R].

    Parameters
    ----------
    path_in         : percorso del file stereo (wav, flac, mp3,…)
    sr              : sample rate di lavoro
    n_fft, hop      : parametri STFT
    harmonic_filt   : lunghezza filtro orizzontale (frames)
    percussive_filt : lunghezza filtro verticale (bin freq)
    mask            : "soft" o "binary"
    power           : esponente per lo spettro (1=magn, 2=potenza)
    out_harm/out_perc : percorsi opzionali per salvare gli stem stereo

    Returns
    -------
    y_h, y_p : np.ndarray
        Array shape=(2, n_samples): [canale_L, canale_R] separati.
    """
    # 1. carica stereo
    y, _ = librosa.load(path_in, sr=sr, mono=False)  # shape (2, n)
    if y.ndim == 1:
        # se mono, duplichiamo per mantenere shape coerente
        y = np.vstack([y, y])
    n_channels, n_samples = y.shape

    # container per gli stem
    y_h = np.zeros_like(y)
    y_p = np.zeros_like(y)

    # 2–6. per ciascun canale, esegui la pipeline HPSS
    for ch in range(n_channels):
        X = librosa.stft(y[ch], n_fft=n_fft, hop_length=hop, window="hann")
        mag = np.abs(X)
        Y = mag**power

        # median filtering anisotropo
        Y_h = scipy.ndimage.median_filter(Y, size=(1, harmonic_filt))
        Y_p = scipy.ndimage.median_filter(Y, size=(percussive_filt, 1))

        # creazione maschere
        if mask == "binary":
            Mh = (Y_h >= Y_p).astype(np.float32)
            Mp = 1.0 - Mh
        else:  # soft
            eps = 1e-10
            denom = Y_h + Y_p + eps
            Mh = Y_h / denom
            Mp = Y_p / denom

        # applicazione maschere e ricostruzione
        X_h = X * Mh
        X_p = X * Mp
        y_h[ch] = librosa.istft(X_h, hop_length=hop, window="hann", length=n_samples)
        y_p[ch] = librosa.istft(X_p, hop_length=hop, window="hann", length=n_samples)

    return y_h, y_p



In [ ]:
# Call the function and save the two components
harmonic_comp, percussive_comp = hpss_stereo(
    path_in=mixture_path,
    sr=44100,
    n_fft=2048,
    hop=512,
    harmonic_filt=31,
    percussive_filt=31,
    mask="soft",
    power=2.0
)

# Play the separated components
print("Percussive Component:")
print(percussive_comp.shape)
display(ipd.Audio(percussive_comp, rate=44100))
print("Harmonic Component:")
display(ipd.Audio(harmonic_comp, rate=44100))

In [ ]:
harmonic_comp, percussive_comp = hpss_stereo(
    path_in=mixture_path,
    sr=44100,
    n_fft=2048,
    hop=512,
    harmonic_filt=51,
    percussive_filt=101,
    mask="soft",
    power=2.0
)

# Play the separated components
print("Percussive Component:")
print(percussive_comp.shape)
display(ipd.Audio(percussive_comp, rate=44100))
print("Harmonic Component:")
display(ipd.Audio(harmonic_comp, rate=44100))

**REPET_SIM VOCAL EXTRACTION**

In [ ]:
import numpy as np
import librosa
import scipy.ndimage
import soundfile as sf
from pathlib import Path

def repet_separate_stereo(
    y: str,
    *,
    sr: int = 44100,
    n_fft: int = 2048,
    hop_length: int = 512,
    mask_type: str = "soft",  # "soft" or "binary"
) -> tuple[np.ndarray, np.ndarray]:
    """
    Separate vocals and music from a stereo mix using the REPET algorithm,
    preserving stereo image by computing mask on the mid-channel and
    applying smoothed channel-specific masks to L and R.

    Parameters
    ----------
    path_in : str
        Path to input stereo audio file.
    sr : int
        Sampling rate for loading and processing.
    n_fft : int
        FFT window size for STFT.
    hop_length : int
        Hop length between STFT frames.
    mask_type : {"soft", "binary"}
        Type of mask: "soft" for proportional, "binary" for hard.
    out_vocal : str | None
        Optional path to save separated vocal track (WAV), stereo.
    out_music : str | None
        Optional path to save separated music track (WAV), stereo.

    Returns
    -------
    vocals : np.ndarray
        Separated vocal signal, shape=(2, n_samples).
    music  : np.ndarray
        Separated music signal, shape=(2, n_samples).
    """
    # 1. Load stereo audio
    # y, _ = librosa.load(path_in, sr=sr, mono=False)
    #display(ipd.Audio(y, rate=sr))
    if y.ndim == 1:
        y = np.vstack([y, y])
    n_ch, n_samples = y.shape

    # 2. Compute mid-channel for mask estimation
    y_mid = np.mean(y, axis=0)

    # 3. STFT on mid-channel
    X_mid = librosa.stft(y_mid, n_fft=n_fft, hop_length=hop_length, window='hann', center=True)
    V_mid = np.abs(X_mid) ** 2

    # 4. Estimate repeating period using onset envelope
    onset_env = librosa.onset.onset_strength(y=y_mid, sr=sr, hop_length=hop_length)
    tempos = librosa.beat.tempo(onset_envelope=onset_env, sr=sr, hop_length=hop_length)
    if len(tempos) == 0:
        raise ValueError("Could not estimate tempo. Check audio quality.")
    tempo = float(tempos[0])
    period = int(round(60 * sr / (tempo * hop_length)))
    if period < 1:
        raise ValueError("Estimated period too small. Check audio or parameters.")
    print(f"Estimated tempo: {tempo:.2f} BPM, repeating period: {period} frames")

    # 5. Build repeating pattern W
    n_bins, n_frames = V_mid.shape
    n_segments = n_frames // period
    if n_segments < 2:
        raise ValueError("Not enough repeating segments. Try a smaller period or longer audio.")
    V_trim = V_mid[:, :n_segments * period]
    segments = V_trim.reshape(n_bins, period, n_segments)
    W = np.median(segments, axis=2)

    # 6. Expand W to full length
    W_full = np.tile(W, (1, n_segments))
    tail = n_frames - W_full.shape[1]
    if tail > 0:
        W_full = np.hstack((W_full, W[:, :tail]))

    # 7. Separate per channel with smoothing
    music = np.zeros((n_ch, n_samples))
    vocals = np.zeros((n_ch, n_samples))
    eps = 1e-10
    for ch in range(n_ch):
        # STFT per channel
        X = librosa.stft(y[ch], n_fft=n_fft, hop_length=hop_length, window='hann', center=True)
        V = np.abs(X)**2

        # Mask creation per channel
        if mask_type == "binary":
            M = (V >= W_full).astype(float)
        else:
            M = W_full / (V + eps)
        M = np.clip(M, 0, 1)

        # Smoothing mask to reduce artefacts
        mask_amp = np.sqrt(M)
        mask_amp = scipy.ndimage.median_filter(mask_amp, size=(3, 3))

        # Apply mask and reconstruct music
        X_music = X * mask_amp
        y_music = librosa.istft(X_music, hop_length=hop_length, window='hann', center=True, length=n_samples)
        music[ch] = y_music
        vocals[ch] = y[ch] - y_music


    return vocals, music



In [ ]:
vocal, music = repet_separate_stereo(
    y=mixture,
    sr=44100,
    n_fft=2048,
    hop_length=512,
    mask_type="soft"
)
other = mixture - vocal - music
print("Mixture:")
display(ipd.Audio(mixture, rate=44100))
print("Difference between original and music component:")
display(ipd.Audio(other, rate=44100))

# output_path = Path("/Users/alessandromanattini/Desktop/MAE/SELECTED TOPIC/PROJECT STMAE")
# output_other= output_path / "other.wav"
# sf.write(output_other, other.T, 44100)

# _, drums = hpss_stereo(
#     path_in=output_other,
#     sr=44100,
#     n_fft=2048,
#     hop=512,
#     harmonic_filt=31,
#     percussive_filt=31,
#     mask="soft",
#     power=2.0
# )

# print("Drums Component:")

# display(ipd.Audio(drums, rate=44100))

# Play the separated components
print("Vocal Component:")
display(ipd.Audio(vocal, rate=44100))
print("Music Component:")
display(ipd.Audio(music, rate=44100))

HPSS+REPET-SIM



In [ ]:
# Call the function and save the two components
harmonic_comp, percussive_comp = hpss_stereo(
    path_in=mixture_path,
    sr=44100,
    n_fft=2048,
    hop=512,
    harmonic_filt=31,
    percussive_filt=31,
    mask="soft",
    power=2.0
)

# Save harmonic component in this folder
output_path = Path("/Users/alessandromanattini/Desktop/MAE/SELECTED TOPIC/PROJECT STMAE/musdb18hq_trimmed")
output_harmonic = output_path / "harmonic.wav"
sf.write(output_harmonic, harmonic_comp.T, 44100)

vocal, music = repet_separate_stereo(
    path_in=output_harmonic,
    sr=44100,
    n_fft=2048,
    hop_length=512,
    mask_type="soft"
)



# Play the separated components
print("Percussive Component:")

display(ipd.Audio(percussive_comp, rate=44100))
print("Vocal Component:")
display(ipd.Audio(vocal, rate=44100))
print("Music Component:")
display(ipd.Audio(music, rate=44100))